<a href="https://colab.research.google.com/github/DaraRahma536/TensorFlow-in-Action/blob/main/Chapter_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Chapter 10: Natural Language Processing with TensorFlow – Language Modeling**
Bab ini membahas pemodelan bahasa (language modeling) menggunakan TensorFlow dengan model GRU (Gated Recurrent Unit). Tujuannya adalah untuk memprediksi kata berikutnya berdasarkan urutan kata sebelumnya, yang kemudian dapat digunakan untuk membuat teks baru (misalnya, cerita anak-anak). Dataset yang digunakan adalah bAbI dataset dari Facebook, berisi cerita anak-anak.

# **Langkah-langkah Utama:**
# **1. Memproses Data**
---
* Dataset: CBTest.tgz dari bAbI dataset.
* Preprocessing:
-&nbsp;Membaca cerita dari file teks.  
-&nbsp;Menggunakan n-grams (bigrams) untuk mengurangi ukuran vocabulary.  
-&nbsp;Tokenization dengan Tokenizer dari Keras.  
* Pipeline Data dengan TensorFlow:
-&nbsp;Menggunakan tf.data.Dataset.window() untuk membuat window dari urutan teks.
-&nbsp;Memisahkan input dan target (target adalah input yang digeser satu posisi ke kanan).

# **2. Model GRU untuk Language Modeling**
---
Arsitektur Model:
* Embedding Layer untuk merepresentasikan n-grams sebagai vektor.
* GRU Layer dengan 1024 unit.
* Dense Layer dengan 512 unit + ReLU.
* Dense Layer akhir dengan ukuran vocabulary + softmax.

GRU vs LSTM: GRU lebih sederhana dengan dua gate (reset dan update) dan satu state, namun performa setara dengan LSTM.

# **3. Metrik Evaluasi: Perplexity**
---
* Perplexity mengukur seberapa "terkejut" model melihat kata target diberikan urutan sebelumnya.
* Perhitungan: Perplexity = exp(mean(cross_entropy_loss)).
* Implementasi: Membuat custom metric PerplexityMetric dengan subclass tf.keras.metrics.Mean.

# **4. Training dan Evaluasi Model**
---
* Training dengan callbacks: CSVLogger, ReduceLROnPlateau, EarlyStopping.
* Hasil: Perplexity validasi ≈ 9.5, akurasi ≈ 45.7% pada test set.
* Penyimpanan model: Model disimpan dalam format .h5.

# **5. Generasi Teks dengan Greedy Decoding**
---
* Inference Model: Membuat model khusus untuk generasi teks dengan functional API.
* Proses Greedy Decoding:
-&nbsp;Mulai dengan seed teks.
-&nbsp;Prediksi kata berikutnya secara rekursif, menggunakan kata yang diprediksi sebagai input berikutnya.
* Hasil: Teks yang dihasilkan cukup baik tetapi terdapat kesalahan tata bahasa dan ejaan.

# **6. Peningkatan dengan Beam Search**
---
* Beam Search: Memprediksi beberapa langkah ke depan dan memilih urutan dengan probabilitas gabungan tertinggi.
* Parameter: beam_width (jumlah kandidat per langkah) dan beam_depth (jumlah langkah pencarian).
* Implementasi: Fungsi rekursif untuk menjelajahi ruang pencarian.
* Hasil: Teks yang dihasilkan lebih koheren dan gramatikal dibanding greedy decoding.

# **Kode Utama yang Direproduksi:**
---
### **1. Preprocessing dan Tokenization**

In [ ]:
def get_ngrams(text, n):
    return [text[i:i+n] for i in range(0, len(text), n)]

train_ngram_stories = [get_ngrams(s, ngrams) for s in stories]
tokenizer = Tokenizer(num_words=n_vocab, oov_token='unk', lower=False)
tokenizer.fit_on_texts(train_ngram_stories)
train_data_seq = tokenizer.texts_to_sequences(train_ngram_stories)

### **2. Pipeline Data dengan Window**

In [ ]:
def get_tf_pipeline(data_seq, n_seq, batch_size=64, shift=1, shuffle=True):
    text_ds = tf.data.Dataset.from_tensor_slices(tf.ragged.constant(data_seq))
    if shuffle:
        text_ds = text_ds.shuffle(buffer_size=len(data_seq)//2)

    text_ds = text_ds.flat_map(
        lambda x: tf.data.Dataset.from_tensor_slices(x)
        .window(n_seq+1, shift=shift)
        .flat_map(lambda window: window.batch(n_seq+1, drop_remainder=True))
    )

    if shuffle:
        text_ds = text_ds.shuffle(buffer_size=10*batch_size)

    text_ds = text_ds.batch(batch_size)
    text_ds = tf.data.Dataset.zip(
        text_ds.map(lambda x: (x[:,:-1], x[:, 1:]))
    ).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return text_ds

### **3. Model GRU untuk Language Modeling**

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(input_dim=n_vocab+1, output_dim=512, input_shape=(None,)),
    tf.keras.layers.GRU(1024, return_state=False, return_sequences=True),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(n_vocab, name='final_out'),
    tf.keras.layers.Activation(activation='softmax')
])
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy', PerplexityMetric()]
)

### **4. Custom Metric: Perplexity**

In [ ]:
class PerplexityMetric(tf.keras.metrics.Mean):
    def __init__(self, name='perplexity', **kwargs):
        super().__init__(name=name, **kwargs)
        self.cross_entropy = tf.keras.losses.SparseCategoricalCrossentropy(
            from_logits=False, reduction='none'
        )

    def _calculate_perplexity(self, real, pred):
        loss_ = self.cross_entropy(real, pred)
        mean_loss = K.mean(loss_, axis=-1)
        perplexity = K.exp(mean_loss)
        return perplexity

    def update_state(self, y_true, y_pred, sample_weight=None):
        perplexity = self._calculate_perplexity(y_true, y_pred)
        super().update_state(perplexity)

### **5. Inference Model untuk Generasi Teks**

In [ ]:
inp = tf.keras.layers.Input(shape=(None,))
inp_state = tf.keras.layers.Input(shape=(1024,))

emb_layer = tf.keras.layers.Embedding(input_dim=n_vocab+1, output_dim=512)
emb_out = emb_layer(inp)

gru_layer = tf.keras.layers.GRU(1024, return_state=True, return_sequences=True)
gru_out, gru_state = gru_layer(emb_out, initial_state=inp_state)

dense_layer = tf.keras.layers.Dense(512, activation='relu')
dense_out = dense_layer(gru_out)

final_layer = tf.keras.layers.Dense(n_vocab)
final_out = final_layer(dense_out)
softmax_out = tf.keras.layers.Activation(activation='softmax')(final_out)

infer_model = tf.keras.models.Model(
    inputs=[inp, inp_state], outputs=[softmax_out, gru_state]
)

# Transfer weights from trained model
emb_layer.set_weights(model.get_layer('embedding').get_weights())
gru_layer.set_weights(model.get_layer('gru').get_weights())
dense_layer.set_weights(model.get_layer('dense').get_weights())
final_layer.set_weights(model.get_layer('final_out').get_weights())

### **6. Beam Search untuk Generasi Teks yang Lebih Baik**

In [ ]:
def beam_search(model, input_, state, beam_depth=5, beam_width=3):
    results = []

    def recursive_fn(input_, state, sequence, log_prob, i):
        if i == beam_depth:
            results.append((list(sequence), state, np.exp(log_prob)))
            return
        else:
            output, new_state = model.predict([input_, state])
            top_probs, top_ids = tf.nn.top_k(output, k=beam_width)
            top_probs, top_ids = top_probs.numpy().ravel(), top_ids.numpy().ravel()

            for p, wid in zip(top_probs, top_ids):
                new_log_prob = log_prob + np.log(p)
                if len(sequence) > 0 and wid == sequence[-1]:
                    new_log_prob += np.log(1e-1)
                sequence.append(wid)
                recursive_fn(np.array([[wid]]), new_state, sequence, new_log_prob, i+1)
                sequence.pop()

    recursive_fn(input_, state, [], 0.0, 0)
    results = sorted(results, key=lambda x: x[2], reverse=True)
    return results